# MBTI별 포스팅 모으기 

In [1]:
import os
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request

In [2]:
mbti=pd.read_csv('.\data\\training\\mbti.csv')

# tokenize and stemming

In [3]:
X_data = mbti['posts'].ravel()
y_data = mbti['type']


print("normalize 시작")
normalized_text = []
for sentence in X_data: #for sentence in X_data.ravel()
    rm_urls = re.sub(r'http[s]?://\S+', '', sentence)
    clean_sentence = re.sub('[^A-Za-z\s]+', '', rm_urls.lower())
    normalized_text.append(clean_sentence)
X_data = normalized_text
print("normalize 완료")

normalize 시작
normalize 완료


In [4]:
import nltk
import gc
# nltk.download('punkt')
# nltk.download('wordnet')
from nltk import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
ps=PorterStemmer()

# print("stopwords 제거 시작")
stop_words = set(stopwords.words('english'))
#print("stopwords 제거 완료")

print("stemming 시작")
stemData=[]
for sentence in X_data:
    tokenData = nltk.word_tokenize(sentence)
    tempData = []
    for word in tokenData:
        if word not in stop_words:
            tempData.append(ps.stem(word))
    stemData.append(tempData)
print("stemming 완료")
print(stemData[0])
stemData = np.array(stemData)
#'list' object has no attribute 'lower' 문제 발생
#2차원 리스트인 stemData를 1차원 리스트로
#flat_stem = [item for sublist in stemData for item in sublist]
del [[mbti]]
gc.collect()

stemming 시작
stemming 완료
['i', 'do', 'stuff', 'like', 'disthey', 'no', 'longer', 'becom', 'outlandish', 'onc', 'they', 'start', 'becom', 'realiti', 'i', 'think', 'you', 'also', 'have', 'to', 'alway', 'rememb', 'whi', 'you', 'start', 'to', 'make', 'music', 'in', 'the', 'first', 'place', 'the', 'way', 'it', 'feel', 'or', 'how', 'you', 'make', 'othersim', 'a', 'musician', 'myself', 'but', 'i', 'start', 'record', 'when', 'i', 'wa', 'a', 'junior', 'in', 'high', 'school', 'i', 'start', 'write', 'music', 'that', 'would', 'person', 'appeal', 'to', 'me', 'and', 'honestli', 'it', 'actual', 'work', 'out', 'pretti', 'well', 'sothen', 'again', 'i', 'hate', 'to', 'gener', 'but', 'that', 'wa', 'my', 'experienceboth', 'loldat', 'an', 'esfj', 'wa', 'my', 'last', 'girl', 'but', 'she', 'wa', 'crazi', 'bruh', 'veri', 'on', 'and', 'off', 'i', 'call', 'her', 'out', 'on', 'it', 'becaus', 'it', 'seem', 'like', 'she', 'wa', 'never', 'consist', 'on', 'how', 'she', 'actual', 'felt', 'but', 'it', 'cool', 'doe', 'n

0

# Vectorization (tfidf)

In [11]:
def dummy_fun(doc):
    return doc

tfidf=TfidfVectorizer(
    analyzer='word',
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None
)

tfidf.fit(stemData)
print(tfidf.transform(stemData).toarray())
X = tfidf.transform(stemData)
y = np.array(y_data)


[[0.00073627 0.00497775 0.01191725 ... 0.00053569 0.00023727 0.00324838]
 [0.00073627 0.00497775 0.01191725 ... 0.00053569 0.00023727 0.00324838]
 [0.00073627 0.00497775 0.01191725 ... 0.00053569 0.00023727 0.00324838]
 [0.00073627 0.00497775 0.01191725 ... 0.00053569 0.00023727 0.00324838]
 [0.00073627 0.00497775 0.01191725 ... 0.00053569 0.00023727 0.00324838]]


In [9]:
import os
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request
from collections import Counter

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
'''
X_data = mbti['posts']
X = X_data.values.reshape(-1, 1)

y_data = mbti['type']
y = y_data.values.reshape(-1, 1)'''

Counter(y_data)
X_resampled, y_resampled = SMOTE(random_state=0).fit_sample(X, y)
#format(Counter(y_resampled))
X_resampled #ndarray
Counter(y_resampled)

Counter({'ENFJ': 1832,
         'ENFP': 1832,
         'ENTJ': 1832,
         'ENTP': 1832,
         'ESFJ': 1832,
         'ESFP': 1832,
         'ESTJ': 1832,
         'ESTP': 1832,
         'INFJ': 1832,
         'INFP': 1832,
         'INTJ': 1832,
         'INTP': 1832,
         'ISFJ': 1832,
         'ISFP': 1832,
         'ISTJ': 1832,
         'ISTP': 1832})

# XGBoost_Oversampling

In [10]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from matplotlib import pyplot

param = {}
param['learning_rate'] = 0.03
param['n_estimators'] = 500
param['max_depth'] = 10


print("Model building Start")
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size= 0.3, random_state=1234)
model = XGBClassifier(**param)
model.fit(X_train, y_train)

Model building Start


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.03, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=500, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [11]:
preds = model.predict(X_test)
print("Model building End")

Model building End


In [12]:
import joblib

joblib.dump(model, 'XGBoost_smote_parameters.pkl')

['XGBoost_smote_parameters.pkl']

In [13]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

print("Evaluation Start")
# labels과 guesses
labels = preds
guesses = y_test

print(accuracy_score(labels, guesses))
print(recall_score(labels, guesses, average=None))
print(precision_score(labels, guesses, average=None))
print(f1_score(labels, guesses, average=None))

print("Evaluation END")

Evaluation Start
0.9160791448715033
[0.97012302 0.90757856 0.95978062 0.87921847 0.99415205 0.99620493
 1.         0.98327138 0.77094972 0.73127036 0.8359375  0.8036036
 0.98568873 0.95644283 0.96346154 0.93965517]
[0.97354497 0.87678571 0.95454545 0.87610619 0.99029126 0.98499062
 0.98954704 0.98878505 0.76243094 0.78496503 0.81992337 0.79218472
 0.97868561 0.95644283 0.96905222 0.96631206]
[0.97183099 0.89191644 0.95715588 0.87765957 0.9922179  0.99056604
 0.99474606 0.9860205  0.76666667 0.75716695 0.827853   0.79785331
 0.98217469 0.95644283 0.96624879 0.9527972 ]
Evaluation END


In [14]:
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import cross_val_score

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
score = cross_val_score(model, X_train, y_train, cv=skf )
print(score)
print(score.mean())

KeyboardInterrupt: 

# HarryPotter MBTI

In [7]:
import nltk
import numpy as np
import pandas as pd

from nltk import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

import re
import glob

'''HP data Normalization and Stemming'''
h_normalized_text = []
h_stemData = []

ps=PorterStemmer()
stop_words = set(stopwords.words('english'))

input_file = r'.\data\\test\\'

allHPfile_list = glob.glob(os.path.join(input_file, 'HP_*'))
print(allHPfile_list)

harryPotter={}
for file in allHPfile_list:
    df = pd.read_csv(file, sep=',', encoding='iso-8859-1')
    harryPotter[df['Character'][0]] = df['Sentence'].ravel()

print("stemming 시작")
for element in harryPotter.keys():
    for sentence in harryPotter[element]:
        h_clean_sentence = re.sub('[^A-Za-z\s]+', '', sentence.lower())
        h_normalized_text.append(h_clean_sentence)
        for sentence in h_normalized_text:
            tokenData = nltk.word_tokenize(sentence)
            for word in tokenData:
                if word not in stop_words:
                    h_stemData.append(ps.stem(word))
    harryPotter[element] = h_stemData

print("stemming 완료")

['.\\data\\\\test\\HP_DUMBLEDORE.csv', '.\\data\\\\test\\HP_HAGRID.csv', '.\\data\\\\test\\HP_HARRY.csv', '.\\data\\\\test\\HP_HERMIONE.csv', '.\\data\\\\test\\HP_RON.csv']
stemming 시작
stemming 완료


In [10]:
'''HP data Vectorization'''

hx_stemData = []
hy=[]

for element in harryPotter.keys():
    hx_stemData.append(harryPotter[element])
    hy.append(element)

HX_data = tfidf.transform(hx_stemData)
print(HX_data)
HY_data = np.array(hy)
print(HY_data)


NotFittedError: The TF-IDF vectorizer is not fitted

In [21]:
import joblib  
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from matplotlib import pyplot

model = joblib.load('XGBoost_smote_parameters.pkl')
# xgb_model_loaded.get_xgb_params()
harrypotter_predict = model.predict(HX_data)
print(harrypotter_predict)

['ESFP' 'ESFP' 'ESFP' 'ESFP' 'ESFP']
